# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: LoRA
* Model: distilbert-base-uncased
* Evaluation approach: 
* Fine-tuning dataset: ealvaradob/phishing-dataset

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [1]:
!pip install -U scikit-learn -q
!pip install peft -q

In [2]:
import numpy as np
import torch
from datasets import load_dataset, Dataset
from sklearn.model_selection import train_test_split
import gc
from transformers import AutoModelForSequenceClassification, AutoTokenizer, \
    DataCollatorWithPadding, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType

In [3]:
# verify the compute resource
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"training on: {device}")

training on: cuda


In [30]:
# required function for 
def get_train_test_sets(num_examples, seed=42, test_size=0.2):
    """
    Loads phishing-dataset from Huggingface and splits into train and test sets.
    
    Args:
        seed: (int) ensures reproducable random sample
        num_examples: (int) number of samples to split into train and test
        test_size: (float) fraction of dataset to hold back for evaluation
        
    Returns:
        tuple of train and test datasets as Dataset objects
        
    """
    
    dataset = load_dataset("ealvaradob/phishing-dataset", "combined_reduced", trust_remote_code=True).shuffle(seed=seed)
    display(dataset)
    
    # turn Dataset object into a pandas df in order to get a random sample and use sklearn train_test_split
    df = dataset['train'].to_pandas().sample(n=num_examples, random_state=seed)

    # Delete the original dataset to free up memory
    del dataset

    # Run the garbage collector
    gc.collect()

    # preview df
    display(df.head())
    print(f"num samples: {len(df)}")
    
    # split into train and test sets
    train, test = train_test_split(df, test_size=test_size, shuffle=True, random_state=seed)

    # convert back into Dataset objects
    train, test = Dataset.from_pandas(train, preserve_index=False), Dataset.from_pandas(test, preserve_index=False)
    
    return train, test


def tokenize_dataset(train, test, model_name="distilbert-base-uncased"):
    """return tokenized examples from the dataset"""
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    def preprocess_function(examples):
        """function to map over dataset to tokenize examples"""
        return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt")

    train = train.map(preprocess_function, batched=True)
    test = test.map(preprocess_function, batched=True)
    
    return train, test, tokenizer


def build_model(model_name="distilbert-base-uncased", requires_grad=False, is_lora=False):
    
    model = AutoModelForSequenceClassification.from_pretrained(model_name,
                                                 num_labels=2,
                                                 id2label={0: "benign", 1: "phishing"},
                                                 label2id={"benign": 0, "phishing": 1},
                                                 )
        
    # Ensure LoRA parameters are trainable
    if is_lora:
        # Freeze all the parameters of the base model
        for param in model.base_model.parameters():
            param.requires_grad = requires_grad
            
        for name, param in model.named_parameters():
            if "lora" in name.lower():
                param.requires_grad = True
    else:
        # Freeze all the parameters of the base model
        for param in model.base_model.parameters():
            param.requires_grad = requires_grad
            
        model.classifier

    return model


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


def build_trainer(model, train, test, tokenizer, dir_name, lr, batch_size, epochs):
    trainer = Trainer(
        model=model,
        args=TrainingArguments(
            output_dir="./data/" + dir_name,
            # Set the learning rate
            learning_rate = lr,
            # Set the per device train batch size and eval batch size
            per_device_train_batch_size = batch_size,
            per_device_eval_batch_size = batch_size,
            # Evaluate and save the model after each epoch
            evaluation_strategy = 'epoch',
            save_strategy = 'epoch',
            num_train_epochs=epochs,
            weight_decay=0.01,
            load_best_model_at_end=True,
        ),
        train_dataset=train,
        eval_dataset=test,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics,
    )
    return trainer

In [6]:
# load dataset and get train/test splits
train, test = get_train_test_sets(num_examples=500)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 77677
    })
})

,text,label
77156,https://warriorplus.com/support/admin/password...,0
28465,mlssoccer.com/videos?id=21318,0
12465,Robert Harley writes:\n> Chuck Murcko wrote:> ...,0
39444,On the topic ofIt is time to refinance!Your cr...,1
14496,<!doctypehtml><html ng-app=app ng-strict-di><t...,1


num samples: 500


In [7]:
# view an example
sample = train[2]['text']
print(f"sample text: {sample}")

# check length of longest text string although sequence length will be capped by model limit.
max_length = 0

for i in train:
    if len(i['text']) > max_length:
        max_length = len(i['text'])
    
print(f"length of longest text string: {max_length}")

sample text: saliturocarpet.com/wp-content/plugins/akismet/item/s
length of longest text string: 86153


### Tokenize the Data

In [16]:
# tokenize the train and test splits
train, test, tokenizer = tokenize_dataset(train, test)

# Show first example of tokenized training set
print(train[0]['input_ids'])

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

[101, 1026, 999, 1011, 1011, 2303, 1063, 4281, 1011, 3746, 1024, 24471, 2140, 1006, 8299, 1024, 1013, 1013, 4871, 1012, 12625, 26745, 4168, 1012, 4012, 1013, 4871, 1013, 3277, 1013, 2327, 1011, 2157, 1012, 21025, 2546, 1007, 1025, 3609, 1024, 1001, 27533, 2546, 14142, 1065, 1037, 1024, 4957, 1063, 3609, 1024, 1001, 27533, 2546, 14142, 1025, 3793, 1011, 11446, 1024, 2104, 4179, 1025, 15489, 1011, 3635, 1024, 3671, 1065, 1037, 1024, 4716, 1063, 3609, 1024, 1001, 27533, 2546, 14142, 1025, 3793, 1011, 11446, 1024, 2104, 4179, 1025, 15489, 1011, 3635, 1024, 3671, 1065, 1037, 1024, 3161, 1063, 3609, 1024, 3897, 1025, 3793, 1011, 11446, 1024, 3904, 1065, 1037, 1024, 25215, 2099, 1063, 3609, 1024, 3897, 1025, 3793, 1011, 11446, 1024, 3904, 1065, 1052, 1012, 2516, 1063, 4281, 1024, 1001, 1039, 2575, 2683, 15878, 2546, 1025, 3675, 1011, 3953, 1024, 1001, 1018, 2546, 2475, 2546, 2549, 2050, 1018, 2361, 2595, 5024, 1025, 3675, 1011, 2187, 1024, 1001, 1041, 2581, 2094, 2629, 2063, 2549, 1018, 2361,

### Load and setup the model

In [13]:
model = build_model()
print(model)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [17]:
# use the huggingface trainer class to train the model
trainer = build_trainer(model, train, test, tokenizer, dir_name='phishing_or_benign', 
                        lr=2e-5, batch_size=16, epochs=1)
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.685513,0.520000


Checkpoint destination directory ./data/phishing_or_benign/checkpoint-25 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=25, training_loss=0.6840260314941407, metrics={'train_runtime': 10.3652, 'train_samples_per_second': 38.591, 'train_steps_per_second': 2.412, 'total_flos': 52986959462400.0, 'train_loss': 0.6840260314941407, 'epoch': 1.0})

In [18]:
trainer.evaluate()

{'eval_loss': 0.6855133771896362,
 'eval_accuracy': 0.52,
 'eval_runtime': 1.5741,
 'eval_samples_per_second': 63.529,
 'eval_steps_per_second': 4.447,
 'epoch': 1.0}

## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [19]:
lora_train, lora_test = get_train_test_sets(num_examples=500)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 77677
    })
})

,text,label
77156,https://warriorplus.com/support/admin/password...,0
28465,mlssoccer.com/videos?id=21318,0
12465,Robert Harley writes:\n> Chuck Murcko wrote:> ...,0
39444,On the topic ofIt is time to refinance!Your cr...,1
14496,<!doctypehtml><html ng-app=app ng-strict-di><t...,1


num samples: 500


In [20]:
lora_train, lora_test, lora_tokenizer = tokenize_dataset(lora_train, lora_test)

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [27]:
lora_train

Dataset({
    features: ['text', 'label', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 400
})

In [25]:
lora_model = build_model(is_lora=True)


# lora_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased",
#                                              num_labels=2,
#                                              id2label={0: "benign", 1: "phishing"},
#                                              label2id={"benign": 0, "phishing": 1},
#                                              )

# # Freeze all the parameters of the base model
# for param in model.base_model.parameters():
#     param.requires_grad = False

# # Ensure LoRA parameters are trainable
# for name, param in lora_model.named_parameters():
#     if "lora" in name.lower():
#         param.requires_grad = True
    
# lora_model.classifier

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:

# create a PEFT config
config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,
    lora_alpha=32,
    lora_dropout=0.10,
    bias="none",
    target_modules=["q_lin", "k_lin", "v_lin", "out_lin", "ffn.lin1", "ffn.lin2"] #can only apply LoRA to linear layers
)

# create a PEFT model
lora_model = get_peft_model(lora_model, config)

In [32]:
lora_trainer = build_trainer(lora_model, lora_train, lora_test, 
                             lora_tokenizer, dir_name='lora_phishing_or_benign', lr=2e-5, batch_size=16, epochs=2)

In [33]:
# Start training
lora_trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.685273,0.520000
2,No log,0.682780,0.520000


Checkpoint destination directory ./data/lora_phishing_or_benign/checkpoint-25 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./data/lora_phishing_or_benign/checkpoint-50 already exists and is non-empty.Saving will proceed but saved results may be invalid.


TrainOutput(global_step=50, training_loss=0.6659337615966797, metrics={'train_runtime': 39.9672, 'train_samples_per_second': 20.016, 'train_steps_per_second': 1.251, 'total_flos': 109059883008000.0, 'train_loss': 0.6659337615966797, 'epoch': 2.0})

In [34]:
lora_trainer.evaluate()

{'eval_loss': 0.6827802062034607,
 'eval_accuracy': 0.52,
 'eval_runtime': 1.9532,
 'eval_samples_per_second': 51.198,
 'eval_steps_per_second': 3.584,
 'epoch': 2.0}

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.